In [1]:
API_KEY = "IO0O0xKDel8d6VvHsPTDAgjY8AX8zscylsYxfg00"

import pandas as pd
import requests

In [2]:
#Task 1: Fetch Basic Stock Data

def get_stock_data():
    symbols = ["MSFT", "AAPL", "NVDA"]  # Microsoft, Apple, Nvidia Stock
    url = "https://yfapi.net/v6/finance/quote"
    headers = {"x-api-key": API_KEY}
    params = {"symbols": ",".join(symbols)}

    response = requests.get(url, headers=headers, params=params).json()

    for stock in response["quoteResponse"]["result"]:
        print(f"Stock Ticker: {stock['symbol']}, "
              f"Company: {stock.get('shortName', 'N/A')}, "
              f"Current Market Price: ${stock.get('regularMarketPrice', 'N/A'):.2f}")


get_stock_data()

Stock Ticker: MSFT, Company: Microsoft Corporation, Current Market Price: $399.73
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36
Stock Ticker: NVDA, Company: NVIDIA Corporation, Current Market Price: $131.28


In [3]:
#Task 2.1: Fetch Additional Data Using Modules Dynamically

def get_additional_data():

    symbols = ["MSFT", "AAPL", "NVDA"]  # Fixed stocks
    module_choice = input("Choose a module (summaryDetail for 52W High/Low, financialData for ROA): ").strip()

    stock_data = []  # Storing stock details

    for symbol in symbols:
        url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
        headers = {"x-api-key": API_KEY}
        params = {"modules": module_choice}

        response = requests.get(url, headers=headers, params=params).json()

        result = response["quoteSummary"]["result"][0].get(module_choice, {})

        # Extract necessary fields according to selected module
        if module_choice == "summaryDetail":
            stock_data.append({
                "Stock": symbol,
                "52 Week High": result.get("fiftyTwoWeekHigh", {}).get("raw", "N/A"),
                "52 Week Low": result.get("fiftyTwoWeekLow", {}).get("raw", "N/A")
            })
        elif module_choice == "financialData":
            stock_data.append({
                "Stock": symbol,
                "Return on Assets (ROA)": f"{result.get('returnOnAssets', {}).get('raw', 'N/A'):.2%}"
                if result.get('returnOnAssets', {}).get('raw', "N/A") != "N/A" else "N/A"
            })

    # Converting list to DataFrame to display
    df = pd.DataFrame(stock_data)
    print("\nStock Data:")
    display(df)

# Run the function
get_additional_data()


Choose a module (summaryDetail for 52W High/Low, financialData for ROA): financialData

Stock Data:


,Stock,Return on Assets (ROA)
0,MSFT,14.65%
1,AAPL,22.52%
2,NVDA,59.13%


In [5]:
#Task 2.2: Fetch Additional Data Using Modules

def get_trending_stocks():

    url = f"https://yfapi.net/v1/finance/trending/US"  # Fetching US trending stocks
    headers = {"x-api-key": API_KEY}

    response = requests.get(url, headers=headers).json()

    trending_symbols = [stock["symbol"] for stock in response["finance"]["result"][0]["quotes"][:5]]  # Top 5
    print(f"Fetching additional details for trending stocks: {', '.join(trending_symbols)}")

    # Fetch additional data
    url = f"https://yfapi.net/v6/finance/quote"
    params = {"symbols": ",".join(trending_symbols)}

    response = requests.get(url, headers=headers, params=params).json()

    # Display details for each stock: 52 Week High/Low & Current Price
    for stock in response["quoteResponse"]["result"]:
        print(f"\nStock: {stock['symbol']} ({stock.get('shortName', 'N/A')})")
        print(f"Current Price: ${stock.get('regularMarketPrice', 'N/A')}")
        print(f"52 Week High: ${stock.get('fiftyTwoWeekHigh', 'N/A')}")
        print(f"Week Low: ${stock.get('fiftyTwoWeekLow', 'N/A')}")


get_trending_stocks()


Fetching additional details for trending stocks: NVDA, SNOW, CRM, IONQ, BYND

Stock: NVDA (NVIDIA Corporation)
Current Price: $131.28
52 Week High: $153.13
Week Low: $75.606

Stock: SNOW (Snowflake Inc.)
Current Price: $166.19
52 Week High: $233.88
Week Low: $107.13

Stock: CRM (Salesforce, Inc.)
Current Price: $307.33
52 Week High: $369.0
Week Low: $212.0

Stock: IONQ (IonQ, Inc.)
Current Price: $29.93
52 Week High: $54.74
Week Low: $6.22

Stock: BYND (Beyond Meat, Inc.)
Current Price: $3.56
52 Week High: $12.12
Week Low: $3.3


In [8]:
#BONUS Challenge 1

def format_currency(value):
    #Format large currency values:Formating as $1,500,000.00
    if isinstance(value, (int, float)):
        return f"${value:,.2f}"
    return "N/A"

def get_dynamic_module_data():

    symbols = ["MSFT", "AAPL", "NVDA"]  # Stocks to check
    module_choice = input("Enter the module name (summaryDetail, financialData, defaultKeyStatistics): ").strip()

    stock_data = []

    for symbol in symbols:
        url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
        headers = {"x-api-key": API_KEY}
        params = {"modules": module_choice}

        response = requests.get(url, headers=headers, params=params).json()

        result = response["quoteSummary"]["result"][0].get(module_choice, {})

        # Convert all values to strings & format currency where applicable
        formatted_data = {key: format_currency(value.get("raw", "N/A")) if isinstance(value, dict) else value
                          for key, value in result.items()}
        formatted_data["Stock"] = symbol  # Add stock symbol

        stock_data.append(formatted_data)

    # Convert to DataFrame
    df = pd.DataFrame(stock_data)
    print("\nStock Data:")
    display(df)


get_dynamic_module_data()


Enter the module name (summaryDetail, financialData, defaultKeyStatistics): defaultKeyStatistics

Stock Data:


,maxAge,priceHint,enterpriseValue,forwardPE,profitMargins,floatShares,sharesOutstanding,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,...,lastCapGain,annualHoldingsTurnover,latestFundingDate,latestAmountRaised,latestImpliedValuation,latestShareClass,leadInvestor,fundingToDate,totalFundingRounds,Stock
0,1,$2.00,"$2,989,335,511,040.00",$26.57,$0.35,"$7,423,723,340.00","$7,433,979,904.00","$62,492,832.00","$62,178,630.00","$1,735,603,200.00",...,N/A,N/A,N/A,N/A,N/A,None,None,N/A,N/A,MSFT
1,1,$2.00,"$3,754,076,930,048.00",$29.17,$0.24,"$14,998,187,904.00","$15,022,100,480.00","$124,917,523.00","$157,008,120.00","$1,735,603,200.00",...,N/A,N/A,N/A,N/A,N/A,None,None,N/A,N/A,AAPL
2,1,$2.00,"$3,072,906,756,096.00",$29.61,$0.56,"$23,504,522,400.00","$24,490,000,384.00","$271,436,938.00","$287,895,394.00","$1,735,603,200.00",...,N/A,N/A,N/A,N/A,N/A,None,None,N/A,N/A,NVDA


In [11]:
# Tail numbers for tracking
tail_numbers = ["N272BG","N628TS"]# Elon's 2 jets

def get_flight_info(tail_numbers):

    url = "https://opensky-network.org/api/states/all"  # OpenSky API Endpoint
    response = requests.get(url).json()

    # Create a list to store flight details
    flight_data = []

    # Loop through each aircraft in the response
    for aircraft in response["states"]:
        callsign = aircraft[1].strip() if aircraft[1] else "N/A"
        icao24 = aircraft[0]  # Unique aircraft identifier
        lat, lon, altitude, velocity = aircraft[6], aircraft[5], aircraft[7], aircraft[9]

        # Check if aircraft matches our target tail number
        for tail in tail_numbers:
            if callsign == tail:
                flight_data.append({
                    "Aircraft": tail,
                    "Latitude": lat,
                    "Longitude": lon,
                    "Altitude (meters)": altitude,
                    "Speed (m/s)": velocity,
                    "ICAO24": icao24
                })

    # Convert to Pandas DataFrame to display result
    df = pd.DataFrame(flight_data)

    if df.empty:
        print("No matching flights found. Aircraft may be grounded.")
    else:
        print("\nFlight Information:")
        display(df)

get_flight_info(tail_numbers)

No matching flights found. Aircraft may be grounded.
